# Simulation Settings

In [ ]:
import os
import numpy as np
from cideMOD import (
    CellParser,
    SolverCrashed,
    init_results_folder,
    ModelOptions,
    CSI
)

In [ ]:
overwrite = False
case = "Ai_2020"
data_path = f"../../data/data_{case}"
params = "params_LAM.json"

model_options = ModelOptions(mode='P2D', clean_on_exit=False, solve_LAM=True)

In [ ]:
cycling = {
            "name": "Discharge Cycle",
            "type": "Cycle",
            "count": 2,
            "steps": [
                {
                    "name": "Discharge",
                    "type": "Current", 
                    "value": -1, 
                    "unit": "C", 
                    "t_max": {"value": 2, "unit": "h"},
                    "store_delay": -1 ,
                    "min_step": 10,
                    "events": [
                        {
                            "type": "Voltage",
                            "value": 3,  
                            "unit": "V",  
                            "atol": 1e-4, 
                            "rtol": 1e-3,
                            "goto": "Next" 
                        }
                    ]
                },
                {
                    "name": "Pause",
                    "type": "Rest",
                    "t_max": {"value": 600, "unit": "s"},
                    "store_delay": -1,
                    "min_step": 10
                },
                {
                    "name": "Charge-CC",
                    "type": "Current", 
                    "value": 1, 
                    "unit": "C", 
                    "t_max": {"value": 2, "unit": "h"},
                    "store_delay": -1 ,
                    "min_step": 10,
                    "events": [
                        {
                            "type": "Voltage",
                            "value": 4.2,  
                            "unit": "V",  
                            "atol": 1e-4, 
                            "rtol": 1e-3,
                            "goto": "Next" 
                        }
                    ]
                },
                {
                    "name": "Charge-CV",
                    "type": "Voltage",
                    "value": 4.199,
                    "unit": "V",
                    "t_max": {"value": 600, "unit": "s"},
                    "store_delay": -1,
                    "min_step": 10,
                    "events": [{
                        "type": "Current",
                        "value": 0.02,
                        "unit": "C",  
                        "atol": 1e-4, 
                        "rtol": 1e-3,
                        "goto": "Next" 
                    }]
                },
            ]}

cycling_test_plan = {
    'initial_state': {
        'SOC': 1,
        'exterior_temperature': 298
    },
    'steps': [cycling]
}

In [ ]:
save_path = init_results_folder(
    f'{case}_LAM', overwrite=overwrite, copy_files=[os.path.join(data_path,params), model_options.dict(), cycling_test_plan], filenames = ['params', 'simulation_options', 'test_plan']
)

In [ ]:
csi = CSI(os.path.join(data_path,params), model_options, save_path=save_path, overwrite=overwrite)
csi.read_test_plan(cycling_test_plan)

In [ ]:
csi.run_test_plan()

In [ ]:
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# Get global variables
time = csi.problem.WH.get_global_variable('time')
voltage = csi.problem.WH.get_global_variable('voltage')
eps_s_a0 = csi.problem.WH.get_global_variable('eps_s_a0_avg')
eps_s_c0 = csi.problem.WH.get_global_variable('eps_s_c0_avg')
sigma_h_a0 = csi.problem.WH.get_global_variable('sigma_h_a0_avg')
sigma_h_c0 = csi.problem.WH.get_global_variable('sigma_h_c0_avg')

# Plot voltage vs time
fig1, ax1 = plt.subplots(1, 1, figsize=(5.5, 4), dpi=200)
ax1.plot(time, voltage, "-", label='voltage')
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Voltage [V]")

# Plot eps_s_a0 vs time
fig2, axs = plt.subplots(2, 2, figsize=(5.5, 4), dpi=200)
axs[0,0].plot(time, eps_s_a0, "-", label='eps_s_a0')
axs[0,0].set_xlabel("Time [s]")
axs[0,0].set_ylabel(r"$\varepsilon_{s}^a \quad \left[-\right]$")

# Plot eps_s_c0 vs time
axs[0,1].plot(time, eps_s_c0, "-", label='eps_s_c0')
axs[0,1].set_xlabel("Time [s]")
axs[0,1].set_ylabel(r"$\varepsilon_{s}^c \quad \left[-\right]$")

# Plot sigma_h_a0 vs time
axs[1,0].plot(time, sigma_h_a0, "-", label='sigma_h_a0')
axs[1,0].set_xlabel("Time [s]")
axs[1,0].set_ylabel(r"$\sigma_{h}^a \quad \left[Pa\right]$")

# Plot sigma_h_c0 vs time
axs[1,1].plot(time, sigma_h_c0, "-", label='sigma_h_c0')
axs[1,1].set_xlabel("Time [s]")
axs[1,1].set_ylabel(r"$\sigma_{h}^c \quad \left[Pa\right]$")
plt.tight_layout()
plt.show()